## Introduction

#### Motivation

이번 섹션에서는 Backpropagation에 대한 직관적인 이해를 바탕으로 전문적 지식을 더 키우고자 한다. <br />
backpropagation은 네트워크 전체에 대해 반복적인 chain rule을 적용하여 그라디언트를 계산하는 방법 중 하나이다. <br />
backpropagation의 과정과 세부 사항들에 대한 이해는 뉴럴 네트워크를 효과적으로 개발하고, 디자인하고 디버그하는데 매우 중요하다.

#### Problem statement

이번 섹션에서 공부할 핵심 문제는 다음과 같다. <br /> 
주어진 함수 $f(x)$에 대하여 $x$는 입력 값으로 이루어진 벡터이고, 입력 $x$에 대해서 함수 $f$의 그라디언트를 계산하고자 한다. (i.e $\triangledown f(x)$)

#### Motivation

이 문제에 관심을 기울이는 이유에 대해 뉴럴 네트워크의 관점에서 좀 더 구체적으로 살펴 보자. <br />
$f$는 손실 함수($L$)에 해당하고 입력 값 $x$는 학습 데이터와 뉴럴 네트워크의 가중치로 구성되어 있다. <br />
예를 들면, 손실 함수는 SVM Loss 함수가 될 수 있고, 입력 값은 학습 데이터 ($x_i$,$y_i$), $i=1..N$와 가중치, bias (W, b)로 볼 수 있다. <br />
(보통의 기계 학습에서 그러하듯) 여기서 학습 데이터는 미리 주어진 고정된 값으로 볼 수 있고  가중치들은 뉴럴 네트워크의 학습을 위해 실제로 조정되는 값이다. <br />
따라서 입력 예제 $x_i$에 대한 그라디언트를 계산하여 역전파하는 것이 쉽지만, <br /> 
실제로는 일반적으로 파라미터들(e.g. $W, b$)에 대한 그라디언트를 계산하고, 그라디언트 값을 활용하여 파라미터를 업데이트 한다. <br />
하지만, 뉴럴 네트워크가 어떻게 작동하는지 해석하고, 시각화 하는 부분에서는 입력 값 $x_i$에 대한 그라디언트도 유용하게 활용 될 수 있는데, <br /> 
이 부분은 본 강의의 뒷 부분에서 다룰 예정이다.

이미 chain rule을 통해 그라디언트를 도출하는데 익숙할지라도 이 섹션을 간략히 훑어보기를 권장한다. <br /> 
왜냐하면, 이 섹션에서는 다른데서는 보기 힘든 backpropagation에 대한 실제 숫자를 활용한 역방향 흐름에 대해 설명을 할 것이고, <br /> 
이를 통해 얻게 될 통찰력은 이번 강의 전체에 있어 도움이 될 것이라 생각하기 때문이다.

## 그라디언트에 대한 간단한 표현과 이해

복잡한 모델에 대한 수식등을 만들기에 앞서 간단하게 시작을 해보자. <br /> 
$x$와 $y$ 두 숫자의 곱을 계산하는 간단한 함수 $f$를 정의하자. $f(x,y)=xy$. <br /> 
각각의 입력 변수에 대한 편미분은 간단한 수학으로 아래와 같이 구해진다. <br />

<img src="./Images/1.png" width=500 />

#### Interpretation

미분이 시사하는 바를 명심하자. 미분은 입력 변수 부근의 아주 작은(0에 매우 가까운) 변화에 대한 해당 함수 값의 변화량이다.

<img src="./Images/2.png" width=300 />

위에 수식을 기술적인 관점에서 보면, 왼쪽에 있는 분수 기호는 오른쪽 분수 기호와 달리 나누기를 뜻하지는 않는다. <br /> 
대신 연산자 $\frac { d }{ dx } $가 함수 $f$에 적용되어 미분된 함수를 의미 하는 것이다. <br />
위의 수식을 이해하는 가장 좋은 방법은 h가 매우 작으면 함수 $f$는 직선으로 근사 될 수 있고, 미분 값은 그 직선의 기울기를 뜻한다. <br />
다시말해, 만약 $x=4$, $y=-3$이면 $f(x,y)=-12$가 되고, $x$에 대한 편미분 값은 $x\frac { \partial f }{ \partial x } $으로 얻어진다. <br />
이 말인 즉슨, $x$를 아주 조금 증가시키면 전체 함수 값은 이 값의 3배만큼 작아진다는 의미이다. (미분값이 음수이므로) <br />
이 것은 위의 수식을 재구성하면 이와 같이 간단히 보여줄 수 있다.($f(x+h)=f(x)+h\frac { df\left( x \right)  }{ dx } $). <br />
비슷하게, $\frac { \partial f }{ \partial y } = 4$이므로, $y$의 값을 아주 작은 $h$만큼 증가 시킨다면, $4h$ 만큼 전체 함수 값은 증가하게 될 것이다.(이번에는 미분 값이 양수)

> 미분은 각 변수가 해당 값에 대하여 전체 함수의 결과 값에 영향을 미치는 민감도와 같은 개념이다.

앞서 말했듯이, 그라디언트 $\nabla f$는 편미분 값들의 벡터이다. 따라서 수식으로 표현하면 다음과 같다. $\nabla f=[\frac { \partial f }{ \partial x } ,\frac { \partial f }{ \partial y } ]=[y,x]$ <br />
그라디언트가 기술적으로는 벡터일지라도 간단한 표현을 위해 "$X$에 대한 편미분"이라는 정확한 표현 대신 "$X$에 대한 그라디언트"와 같은 표현을 종종 쓰게 될 것이다.

다음과 같은 수식에 대해서도 미분값(그라디언트)을 한번 구해보자

<img src="./Images/3.png" width=500 />

위의 수식에서 볼 수 있듯이, $x$, $y$에 대한 미분은 $x$, $y$값에 관계없이 1이다. 당연히 $x$, $y$값이 증가하면 $f$가 증가하기 때문이다. <br />
그리고 $f$값의 증가율 또한 $x$, $y$값에 관계 없이 일정하다. (앞서 살펴본 곱셈의 경우와 다름). <br /> 
마지막으로 살펴볼 함수는 수업에서 자주 다루었던 $max$ 함수이다. <br />

<img src="./Images/4.png" width=600 />

입력 값에 대하여 더 큰 값의 (서브)그라디언트는 1이고, 그렇지 않은 입력 값의 그라디언트는 0이 된다. <br /> 
직관적으로 보면, $x=4$, $y=24$인 경우 max 값은 4이고, 이 함수는 $y$값에 영향을 받지 않는다. <br />
바꾸어 말하면, $y$값을 아주 작은 값인 $h$ 만큼 증가시키더라도 이 함수의 결과 값은 4로 유지된다. 따라서 그라디언트는 0이다.($y$값의 영향이 없다) <br />
물론 $y$값을 매우 크게 증가시킨다면 (예를 들면, 2 이상으로) 함수 $f$값은 바뀌겠지만, 미분은 이런 큰 변화 값과는 관련이 없다. <br />
미분이라는 것이 본래 그 정의에도 있듯($\lim _{  }{ h\rightarrow 0 }$) 아주 작은 입력 값 변화에 대해서 의미를 갖는 값이기 때문이다.

## Compound expressions with chain rule

이제 $f(x,y,z)=(x+y)z$ 같은 multiple composed 함수의 표현식을 생각해보자. <br />
이 표현식은 여전히 바로 미분하기에 충분히 간단하지만, 이 식에 특별한 접근법을 적용할 것이다. <br />
이는 backpropagation 이면에 있는 직관을 이해하는데 도움이 될 것이다. 특히 이 식이 두 개의 표현식 $q = x+y$와 $f = qz$로 분해될 수 있음에 주목하자. <br />
더욱이, 이전 섹션에서 본 것처럼 두 식에 대한 미분값을 어떻게 따로 따로 계산할지 알고 있다. f는 단지 q와 z의 곱이다. <br />
따라서 $\frac { \partial f }{ \partial q } = z,\quad \frac { \partial f }{ \partial z } = q$ 그리고 $q$는 $x$와 $y$의 합이므로 $\frac { \partial q }{ \partial x } = q$, $\frac { \partial q }{ \partial y } = 1$이다. <br />
하지만 중간 결과값인 $q$에 대한 그라디언트($\frac { \partial f }{ \partial q }$)를 신경쓸 필요가 없다. 대신 궁극적으로 입력 $x$, $y$, $z$에 대한 $f$의 그라디언트에 관심이 있다. <br />
chain rule은 이러한 그라디언트 표현식들을 함께 연결시키는 적절한 방법이 곱하기라는 것이라는 것을 보여준다. <br />
예를 들면, $\frac { \partial f }{ \partial x } = \frac { \partial f }{ \partial q }\frac { \partial q }{ \partial x }$ 와 같이 표현할 수 있다. 실제로 이는 단순히 두 그라디언트를 담고 있는 두 수의 곱셈이다. <br />
아래의 예를 통해 확인해보자.

In [ ]:
# set some inputs
x = -2; y = 5; z = -4

# perform the forward pass
q = x + y # q becomes 3
f = q * z # f becomes -12

# perform the backward pass (backpropagation) in reverse order:
# first backprop through f = q * z
dfdz = q # df/dz = q, so gradient on z becomes 3
dfdq = z # df/dq = z, so gradient on q becomes -4
# now backprop through q = x + y
dfdx = 1.0 * dfdq # dq/dx = 1. And the multiplication here is the chain rule!
dfdy = 1.0 * dfdq # dq/dy = 1

결국 [dfdx, dfdy, dfdz] 변수들로 그라디언트가 표현되는데, 이는 f에 대한 변수 x, y, z의 민감도를 보여준다. <br />
이는 backpropagation의 가장 간단한 예이다. 더 나아가서 보다 간결한 표현법을 사용해서 df 파트를 계속 쓸 필요가 없도록 하고 싶을 것이다. <br />
예를 들어 dfdq 대신에 단순히 dq를 쓰고 항상 그라디언트가 최종 출력에 관한 것이다 가정하는 것이다. <br />
또한 이런 계산은 회로도를 가지고 다음과 같이 시각화 할 수 있다.
___

<img src="./Images/5.png" width=400 />

좌측에 실수 값으로 표현되는 회로는 이 계산에 대한 시각적 표현을 보여준다. <br />
forward pass는 입력부터 출력까지 값을 계산한다.(녹색으로 표시됨) <br />
그러고 나서 backward pass는 back propagation을 수행하는데, 이는 끝에서 시작해서 반복적으로 chain rule을 적용해 회로 입력에 대한 모든 길에서 그라디언트(적색으로 표시)를 계산한다. <br />
그라디언트 값은 회로를 통해 거꾸로 흐르는 것을 볼 수있다.
___

## Back propagation에 대한 직관적인 이해

Back propagation는 광장히 지역적인 프로세스임에 주목하자. 회로도 내의 모든 게이트는 몇개의 입력을 받아들이고 곧 바로 두 가지를 계산할 수 있다. <br />
- 게이트의 출력 값
- 게이트의 출력에 대한 입력들의 지역적 그라디언트 값

여기서 게이트들이 포함된 전체 회로의 세세한 부분을 모르더라도 완전히 독립적으로 값들을 계산할 수 있음을 주목하라. <br />
하지만, 일단 forward pass가 끝나면 backpropagation 과정에서 게이트는 결국 전체 회로의 마지막 출력에서의 출력값에 대한 그라디언트 값에 관하여 학습할 것이다. <br />
Chain rule이 의미하는 바를 생각해보자. <br /> 
게이트는 그라디언트를 가져야하고 이 그라디언트는 일반적으로 이 게이트의 모든 입력에 대해 계산된 모든 그라디언트에 곱해진다.

> Chain rule 덕분에, (각 입력에 대한)extra multiplication은 한 개의 그리고 상대적으로 쓸모없는 게이트를 전체 뉴럴 네트워크와 같은 복잡한 회로의 톱니로 바꿀수 있다.

다시 위의 예제를 통해 이것이 어떻게 동작하는지에 대한 직관을 얻자. <br />
덧셈 게이트는 입력 [-2,5]를 받아 3을 출력한다. <br />
이 게이트는 덧셈 연산을 하고 있기 때문에 두 입력에 대한 게이트의 local 그라디언트 값은 +1이 된다. <br />
회로의 나머지 부분을 거쳐서 최종 출력 값으로 -12가 나온다. Chain rule이 회로에서 반복적으로 backwards를 하는 backward pass 과정동안, <br />
덧셈 게이트(곱셈 게이트의 입력인)는 출력 값에 대한 그라디언트 값이 -4라는 것을 학습한다. <br />
만약 회로가 높은 값을 출력하기를 원하는 것으로 의인화 하면, 이 회로가 덧셈 게이트의 출력 값이 4의 힘으로 낮아지길(음의 부호이기 때문) 원하는 것으로 볼 수 있다. <br />
그라디언트 값을 연결하기 위한 반복을 지속하기 위해, 덧셈 게이트는 이 그라디언트 값을 받아들이고 이를 이것의 입력들에 대한 모든 지역적 그라디언트 값에 곱한다. <br /> ($x$와 $y$에 대한 그라디언트 값이 1*-4 = -4 가 되도록) <br /> 
위 과정을 통해 다음의 원하는 효과가 있다는 사실에 주목하자. <br /> 
만약 $x$, $y$가 (음의 그라디언트 값에 대한 반응으로) 감소한다면, 이 덧셈 게이트의 출력은 감소할 것이고 이는 다시 곱셈 게이트의 출력이 증가하도록 만들 것이다. 

따라서 backpropagation은 보다 큰 최종 출력 값을 얻기 위해, 게이트들이 자신들의 출력이 증가하길 원하는지 또는 감소하길 원하는지(how strongly) 서로 소통(trough the gradient signal)하는 것으로 간주할 수 있다.

## Modularity (시그모이드 예제)

위에서 본 게이트들은 상대적으로 임의로 선택된 것이다. 어떤 종류의 함수도 미분가능하다면 게이트로서 역할을 할 수 있다. <br />
필요한 경우 여러 개의 게이트를 그룹지어서 하나의 게이트로 만들거나, 하나의 함수를 여러개의 게이트로 분해할 수도 있다. <br />
이러한 요점을 보여주는 다른 표현식을 살펴보자.

<img src="./Images/6.png" width=300 />

나중에 수업에서 보겠지만, 이 표현식은 시그모이드 활성화 함수를 사용하는 2차원 뉴런(입력 $x$와 가중치 $w$를 갖는)을 나타낸다. <br />
그러나 지금은 이를 매우 단순하게 $w$,$x$를 입력으로 받아 하나의 단일 숫자를 출력하는 함수 정도로 생각하자. <br />
이 함수는 여러 개의 게이트로 구성된다. 위에서 이미 설명한 게이트들(덧셈, 곱셈, 최대)말고도 네 종류의 게이트가 더 있다.

<img src="./Images/7.png" width=500 />

여기서 $f_c$, $f_a$는 각각 입력을 상수 $c$만큼 이동시키고, 상수 $a$만큼 크기를 조정하는 함수이다. <br />
이 함수들은 덧셈과 곱셈이 기술적으로 특별한 경우에 해당하지만, 여기서는 상수 $c$, $a$에 대한 그라디언트가 필요한 것이기에 (새로운) 단일 게이트로서 소개하고자 한다. <br />
그러면 전체 회로는 다음과 같이 나타난다.
___

<img src="./Images/8.png" width=800 />

시그모이드 활성 함수를 갖는 2차원 뉴런에 대한 예시 회로 <br/ >
입력은 [x0, x1]이고 뉴런의 학습 가능한 가중치들은 [w0, w1, w2]이다. <br />
나중에 살펴보겠지만, 뉴런은 입력을 가지고 내적을 계산하고 이 입력에 대한 활성 함수 출력 값은 0부터 1사이의 범위에 들어가도록 시그모이드 함수에 의해 부드럽게 압착이 된다.
___

위 예제에서 $W$, $X$ 사이의 내적의 결과로 동작하는 함수 appliaction의 긴 체인을 보았다. <br />
이런 연산을 제공하는 함수를 시그모이드 함수 $\sigma (x)$라고 한다. <br />
만약 분자에 1을 더하고 다시 빼는 과정을 거친후에 미분을 한다면 입력에 대한 시그모이드 함수의 미분값은 단순화 할 수 있다.

<img src="./Images/9.png" width=700 />

보이는 것처럼 그라디언트는 단순화고 간단해진다. <br />
예를 들어, 시그모이드 표현은 forward pass 과정에서 입력 1.0을 받아 출력 0.73을 계산한다. <br />
이전에 본 회로가 계산했던 것(위 그림)과 비슷하게 위의 미분은 지역 그라디언트 값이 단순히 $(1-0.73)*0.73\cong 0.2$가 됨을 보여준다. <br />
하나의 단순하고 효율적인 표현식을 이용해(더 적은 수치적인 문제를 갖고) 계산된다. <br />
그러므로 실제 적용에서 그러한 연산들을 단일 게이트로 묶어 주는것은 매우 유용하다고 할 수 있다. <br />
코드에서 이 뉴런에 대한 역전파를 살펴보자

In [ ]:
w = [2,-3,-3] # assume some random weights and data
x = [-1, -2]

# forward pass
dot = w[0]*x[0] + w[1]*x[1] + w[2]
f = 1.0 / (1 + math.exp(-dot)) # sigmoid function

# backward pass through the neuron (backpropagation)
ddot = (1 - f) * f # gradient on dot variable, using the sigmoid gradient derivation
dx = [w[0] * ddot, w[1] * ddot] # backprop into x
dw = [x[0] * ddot, x[1] * ddot, 1.0 * ddot] # backprop into w
# we're done! we have the gradients on the inputs to the circuit

#### 구현 팁: 단계적 backpropagation

위 코드에서 볼 수 있듯이, forward pass를 쉽게 backpropped 되는 단계들로 잘게 분해하는 것은 실전에서 도움이 된다. <br />
예를 들어, 여기서 w와 x사이의 내적의 결과를 담는 중간 변수 dot를 만들었다. <br />
backward pass를 하는 동안 이러한 변수들의 그라디언트 값들을 담은 해당 변수들(e.g. ddot, 궁극적으로는 dw, dx)을 연속해서 계산할 수 있다.(역순으로)

이 섹션에서의 요점은 어떻게 backpropagation이 수행되는 지와 forward 함수의 어느 부분을 게이트로 취급해야 할 지에 대한 세부사항은 편의성 문제라는 것이다. <br />
이는 표현식의 어느 부분들이 쉬운 지역 그라디언트를 가지는지 그리고 가장 적은 양의 코드와 노력으로 연결될 수 있다는 것을 이해하는데 도움이 된다.

## Backprop in practice: 단계적 계산

또 다른 예제를 통해 확인해보자. 다음과 같은 형태의 함수가 있다고 가정하자.

<img src="./Images/10.png" width=300 />

명확히 말하면, 실제 역전파의 좋은 예제라는 사실 외에는 이 함수는 완전히 쓸모가 없으며 따라서 왜 이 함수의 그라디언트를 계산해야 하는지도 명확하지 않다. <br />
만약 우리가 $x_i$ 또는 $y_i$에 관해서 미분을 수행한다면 결국 매우 크고 복잡한 식을 얻게 될 것이다. <br />
하지만 그라디언트를 계산하는 데에는 명확한 함수를 도출해 낼 필요는 없기 때문에 그렇게 미분하는 것은 완전히 불필요하다. <br />
단지 어떻게 이를 계산하는지만 알면 된다. 다음은 어떻게 이런 표현식의 forward pass를 구조화 하는지를 나타낸 것이다.

In [ ]:
x = 3 # example values
y = -4

# forward pass
sigy = 1.0 / (1 + math.exp(-y)) # sigmoid in numerator   #(1)
num = x + sigy # numerator                               #(2)
sigx = 1.0 / (1 + math.exp(-x)) # sigmoid in denominator #(3)
xpy = x + y                                              #(4)
xpysqr = xpy**2                                          #(5)
den = sigx + xpysqr # denominator                        #(6)
invden = 1.0 / den                                       #(7)
f = num * invden # done!                                 #(8)

표현식의 마지막 부분에서 forward pass를 계산했다. 각각이 단순한 표현식들인 다수의 중간 매개 변수들을 포함하는 방식으로 코드를 구조화한 것에 주목하자. <br /> 이 표현식들은 이미 지역 그라디언트 값을 알고 있는 쉬운 표현식들이다. 그러므로 역전파를 계산하는 것은 쉬운 일이다. <br />
forward pass 과정의 모든 변수들 (sigy, num, sigx, xpy, xpysqr, den, invden)에 대해 동일한 변수들을 따라서 backward 할 것이다. <br />
같은 이름의 변수명 앞에 d를 붙인다. 이 변수들은 해당 변수에 대한 회로의 출력의 그라디언트를 가진다. <br /> 
덧붙여 말하자면, backprop에서 모든 단일 조각은 이 표현식에 대한 지역 그라디언트를 계산하고 곱셈 형태로 이 그라디언트 값을 연결하는 과정을 수반할 것이다. <br />
각 행에는 forward pass 과정에서 어느 부분에 해당하는지를 표시한 것이다.

In [ ]:
# backprop f = num * invden
dnum = invden # gradient on numerator                             #(8)
dinvden = num                                                     #(8)
# backprop invden = 1.0 / den 
dden = (-1.0 / (den**2)) * dinvden                                #(7)
# backprop den = sigx + xpysqr
dsigx = (1) * dden                                                #(6)
dxpysqr = (1) * dden                                              #(6)
# backprop xpysqr = xpy**2
dxpy = (2 * xpy) * dxpysqr                                        #(5)
# backprop xpy = x + y
dx = (1) * dxpy                                                   #(4)
dy = (1) * dxpy                                                   #(4)
# backprop sigx = 1.0 / (1 + math.exp(-x))
dx += ((1 - sigx) * sigx) * dsigx # Notice += !! See notes below  #(3)
# backprop num = x + sigy
dx += (1) * dnum                                                  #(2)
dsigy = (1) * dnum                                                #(2)
# backprop sigy = 1.0 / (1 + math.exp(-y))
dy += ((1 - sigy) * sigy) * dsigy                                 #(1)
# done! phew

몇 가지 주의할 점

##### Cache forward pass variables

backward pass를 계산하기 위해 forward pass에서 사용한 변수들을 가지고 있는 것은 정말 유용하다. <br />
실제로 이 변수들을 저장해서 backpropagation동안 이용할 수 있도록 코드를 구성하고 싶을 것이다. <br />
이것이 너무 어려운 일이라면, 이 변수들을 다시 계산할 수 있지만 이는 매우 비효율적인 일이다.

#### Gradient add up at forks

forward 표현식은 변수 x,y의 여러번의 곱셈을 수반하므로, <br /> 
backpropagation을 수행할 때 이 변수들에 대한 그라디언트 값을 축적하기 위해 = 대신 += 를 사용해야 하는 점에 주의해야 한다. <br />
이렇게 하지 않으면 덮어쓰게 된다. 이는 미적분학에 나오는 multivariable chain을 따른다. <br /> 
미적분학에서는 하나의 변수가 회로의 다른 부분들로 가지를 뻗어나가면, 이 변수로 flow back되는 그라디언트들은 더해진다고 나와있다.

## Backward 흐름의 패턴

Backward 흐름 그라디언트에 많은 사례들이 직관적인 수준에서 이해될 수 있다는 점은 흥미롭다. <br />
예를 들면, 뉴럴 네트워크에서 자주 사용되는 세가지 게이트(최대, 곱, 합)들이 모두 backprapagation되는 동안 어떻게 행동하는지 이해하는 것은 매우 간단하다. <br />
___

<img src="./Images/11.png" width=500 />

입력 변수들에 대한 그라디언트를 계산하기 위해 backward pass가 진행되는 동안 backpropagation가 어떻게 수행되는지 직관적으로 보여주는 회로도이다. <br />
합 연산은 모든 입력 변수에 대해 그라디언트를 동등하게 분산시킨다. <br />
Max 연산은 가장 큰 입력 변수에 그라디언트를 보낸다. <br />
곱 게이트는 입력 activations를 받아서 이들을 서로 바꾸고, 게이트의 그라디언트가 곱해진다. 
___

예제로서 위의 흐름도를 살펴보자. 다음을 알 수 있다.

덧셈 게이트는 항상 출력에 대한 그라디언트를 취해서 forward pass가 진행되는 동안 이들의 값이 무엇인지에 관계없이 모든 입력들에 동등하게 분산시킨다. <br />
이것은 덧셈 연산에 대한 지역적 그라디언트가 +1이라는 사실로부터 알 수 있고, 그래서 모든 입력들에 대한 그라디언트는 1이 곱해지기 때문에 출력에 대한 그라디언트와 동일하다. <br />
위의 예제 회도로에서 +게이트는 이들의 입력들에 둘다 2.00이라는 그라디언트를 보낸다.

최대 게이트는 모든 입력 변수에 출력 변수의 그라디언트를 그대로 분산시키는 덧셈 게이트와는 다르다. <br/ >
최대 게이트는 forward pass가 진행되는 동안 가장 큰 값을 갖는 입력 변수 하나에만 그라디언트를 보낸다. <br />
이는 최대 게이트에 대한 지역적 그라디언트가 최대 값에서만 1이고 나머지 값에는 0이기 때문이다. <br />
위의 예제 회도로에서, Max 연산자는 w보다 높은 값을 갖는 z변수에 2의 그라디언트를 보내고, w의 그라디언트는 0이 된다.

곱 게이트를 이해하기는 쉽지 않다. 이들의 지역적 그라디언트는 뒤바뀐 입력 값에다가 체인 룰에 의해 이들의 출력에 대한 그라디언트를 곱한 값이다. <br />
위의 예제에서는, x에 대한 그라디언트는 -4.00x2.00이라서 -8.00이다. 

#### Unintuitive effects and their consequences

만약 곱셈 게이트의 입력 변수 중 하나가 매우 작고 다른 하나는 매우 크다면, 곱셈 게이트는 직관적이지 않은 결과를 보여줄 수 있다. <br />
작은 입력 변수에 대해 상대적으로 큰 그라디언트를 할당하고, 큰 입력변수에 대해서는 작은 그라디언트를 할당할 것이다. <br />
입력 변수들과 가중치들을 내적을 하는(${w}^{T} {x}_{i}$) 선형 분류기를 살펴보자. <br /> 
이는 데이터의 규모가 이 가중치들에 대한 그라디언트의 magnitude에 영향을 갖을 것임을 암시한다. <br />
예를 들어, 만약 우리가 모든 입력 예제 $x_i$에 전처리를 하기 위해 1000을 곱했다면, 이 가중치들에 대한 그라디언트는 1000배가 커질 것이고, <br />
이를 보상하기 위해 학습률을 더 작게 해야 할 것이다. <br />

## Gradient for vectorized operations

위에서는 단일 변수들만 고려했지만, 모든 개념은 행렬과 벡터 연산으로 확장된다. 차원과 전치(transpose) 연산에 주의해야한다.

#### 행렬-행렬 곱셈 그라디언트

가장 까다로운 연산은 행렬과 행렬(모든 행렬-벡터, 벡터-벡터의 일반적인 형태) 곱셈이다.

In [ ]:
# forward pass
W = np.random.randn(5, 10)
X = np.random.randn(10, 3)
D = W.dot(X)

# now suppose we had the gradient on D from above in the circuit
dD = np.random.randn(*D.shape) # same shape as D
dW = dD.dot(X.T) #.T gives the transpose of the matrix
dX = W.T.dot(dD)

팁: 차원 분석을 사용해라! <br />
dW나 dX에 대한 표현식을 기억할 필요는 없다. 왜냐하면 이것들은 차원에 기반해 다시 유도하기 쉽기 때문이다. <br />
예를 들면, 우리는 가중치에 대한 그라디언트 dW가 계산된 후에 W와 크기가 같아야 함을 안다. 그리고 이것은 X와 dD의 행렬 곱에 의존한다. <br />
위에서 보는것 처럼, X는 [10x3]의 크기이고 dD는 [5x3]의 크기일 때 dW와 W가 [5x10]의 모양을 갖추고 싶다면, <br /> 
이를 위한 한 가지 방법은 dD.dot(X.T)이다.

#### Work with small, explicit examples

몇몇 사람들은 어떤 벡터화 연산에 대한 그라디언트 업데이트를 유도하는것에 어려움을 겪을 수 있다. <br />
아주 작은 벡터화 예제를 명시적으로 작성하는 것을 추천한다. <br />
종이에 그라디언트를 유도해라. 그리고 이것을 효율적으로 만들기 위해 패턴을 일반화하고, 벡터화 형식으로 만들어보아라.

Erik Learned-Miller has also written up a longer related document on taking matrix/vector derivatives which you might find helpful. <br /> 
[Find it here.](http://cs231n.stanford.edu/vecDerivs.pdf)

## Summary

- We developed intuition for what the gradients mean, how they flow backwards in the circuit, and how they communicate which part of the circuit should increase or decrease and with what force to make the final output higher.
- We discussed the importance of staged computation for practical implementations of backpropagation. You always want to break up your function into modules for which you can easily derive local gradients, and then chain them with chain rule. Crucially, you almost never want to write out these expressions on paper and differentiate them symbolically in full, because you never need an explicit mathematical equation for the gradient of the input variables. Hence, decompose your expressions into stages such that you can differentiate every stage independently (the stages will be matrix vector multiplies, or max operations, or sum operations, etc.) and then backprop through the variables one step at a time.

### References

- [Automatic differentiation in machine learning: a survey](https://arxiv.org/abs/1502.05767)